### Deep FM

In [ ]:
# binary classification
# 0: negative 1: positive
# 변수 = field, encoding 완료한 변수 개수 = feature 개수
# V, embedding matrix의 col개수(k)는 hyperparameter

#### `breast_cancer_data` import

In [ ]:
from sklearn.datasets import *
data = load_breast_cancer(as_frame=True) 
data
#X, Y = file['data'], file['target']
# X = scaler.fit_transform(X)


{'DESCR': '.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry\n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        worst/largest values) of these features were computed for each image,\n        resulting in 30 features.

In [ ]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [ ]:
data['data']

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
import pandas as pd
cancer = pd.DataFrame(data.data, columns=data.feature_names)
cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [ ]:
data.target

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

In [ ]:
file = load_breast_cancer()
X, Y = file.data, file.target

In [ ]:
X.shape

(569, 30)

In [ ]:
# breast_cancer
ALL_FIELDS = data.feature_names # all col is continuous
CONT_FIELDS = data.feature_names
CAT_FIELDS = list(set(ALL_FIELDS).difference(CONT_FIELDS))

NUM_BIN = 10
BATCH_SIZE = 30
EMBEDDING_SIZE = 5

In [ ]:
# config.py , col의 목록을 연속형/범주형을 구분하여 저장한 리스트 & hyperparameter
ALL_FIELDS = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
             'marital-status', 'occupation', 'relationship', 'race',
             'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'country']
CONT_FIELDS = ['age', 'fnlwgt', 'education-num',
               'capital-gain', 'capital-loss', 'hours-per-week']
CAT_FIELDS = list(set(ALL_FIELDS).difference(CONT_FIELDS))

# Hyper-parameters for Experiment
NUM_BIN = 10
BATCH_SIZE = 256
EMBEDDING_SIZE = 5

#### field index & field dict 만들기

##### 인코딩 이후의 데이터에 대해 각 col이 본래 인코딩 이전에 몇 번째 field에 속했었는지에 대한 정보를 저장한 것이 field_index & field_dict


In [ ]:
## pd.cut(df.col, bins=k) : 동일 길이 k개 범주로 데이터 나누기. 연속형->범주형 만들때 사용.
## pd.get_dummies(data, prefix='nm', prefix_sep='-')
## list.extend('a') : list의 마지막 원소로 a 추가
## itertools.repeat(10, times=4): 10 10 10 10, times 지정한하면 무제한으로 

In [ ]:
!pip install config

In [ ]:
# preprocessing
#import config 
from itertools import repeat
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def get_modified_data(X, all_fields, continuous_fields, categorical_fields, is_bin=False):
    field_dict= dict()
    field_index = []
    X_modifi+Frame()

    for index, col in enumerate(X.columns):
        if col not in all_fields:
            print(f'{col} not included: check your column list.' )
            raise ValueError # 프로그래머가 지정한 예외가 발생하도록 강제

        if col in continuous_fields:
            scaler = MinMaxScaler()

            # contin 변수도 구간화?
            if is_bin: # 10개의 범주로 나누기
                X_bin = pd.cut(scaler.fit_transform(X[[col]]).reshape(-1,), NUM_BIN, labels=False)
                X_bin = pd.Series(X_bin).astype('str')

                X_bin_col = pd.get_dummies(X_bin, prefix=col, prefix_sep='-')
                field_dict[index]=list(X_bin_col.columns)
                field_index.extend(repeat(index, X_bin_col.shape[1]))
                X_modified = pd.concat([X_modified, X_cont_col], axis=1) # 옆으로 병합
            else: 
                X_cont_col = pd.DataFrame(scaler.fit_transform(X[[col]]), columns=[col])
                field_dict[index] = col
                field_index.append(index)
                X_modified = pd.concat([X_modified, X_cont_col], axis=1)
        
        if col in categorical_fields:
            X_cat_col =  pd.get_dummies(X[col], prefix=col, prefix_sep='-')
            field_dict[index] = list(X_cat_col.columns)
            field_index.extend(repeat(index, X_cat_col.shape[1]))
            X_modified = pd.concat([X_modified, X_cat_col], axis=1)

    print('Data Prepared...')
    print(f'X shape: {X_modified.shape}')
    print(f'# of feature : {len(field_index)}')
    print(f'# of field: {len(field_dict)}')

    return field_dict, field_index, X_modified



In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# categorical변수 없으므로 skip

In [ ]:
cancer.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

In [ ]:
get_modified_data(cancer, all_fields, continuous_fields, categorical_fields, is_bin=False)

### Model

### FM Model

In [ ]:
import tensorflow as tf
class FM_layer(tf.keras.layers.Layer):
    def __init__(self, num_feature, num_field, embedding_size, field_index):
        super(FM_layer, self).__init__()
        self.embedding_size = embedding_size # k: embedding vector dim
        self.num_feature = num_feature # f: 원래 feature 개수
        self.num_field = num_field # m: grouped field 개수
        self.field_index = field_index # encoding된 X의 col들이 본래 어디 소속이었는지 

        # Parameters of FM Layer
        # w: 1st order interactions
        # V: 2nd order interactions
        self.w = tf.Variable(tf.random.normal(shape=[num_feature], mean=0, stddev=1),name ='w')
        self.V = tf.Variable(tf.random.normal(shape=(num_field,embedding_size), mean=0, stddev=0.01),name ='V')

    def call(self, inputs):
        x_batch = tf.reshape(inputs, [-1, self.num_feature,1])
        # parameter V를 field_index에 맞게 복사하여 num_feature에 맞게 늘림.
        embeds = tf.nn.embedding_lookup(params = self.V, ids = self.field_index)

        # Deep component에서 쓸 input
        # ( batch_size, num_feature, embedding_size )
        new_inputs = tf.math.multiply(x_batch, embeds)

        # (batch_size, )
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1, keepdims=False)

        # (batch_size, )
        interactions = 0.5 * tf.subtract(tf.square(tf.reduce_sum(new_inputs, [1,2])), tf.reduce_sum(tf.square(new_inputs),[1,2]))

        linear_terms = tf.reshape(linear_terms, [-1,1]) # col vector
        interactions = tf.reshape(interactions,[-1,1])  # col vector

        y_fm = tf.concat([linear_terms, interactions],1)

        return y_fm, new_inputs 




### DeepFM Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
#from layers import FM_layer

tf.keras.backend.set_floatx('float32')

In [ ]:
class DeepFM(tf.keras.Model):
    def __init__(self, num_feature, num_field, embedding_size, field_index):
        super(DeepFM, self).__init__()
        self.embedding_size = embedding_size
        self.num_feature = num_feature
        self.num_field =  num_field
        self.field_index = field_index 

        self.fm_layer = FM_layer(num_feature, num_field, embedding_size, field_index)

        self.layers1 = tf.keras.layers.Dense(units=64, activation='relu')
        self.dropout1 = tf.keras.layers.Dropout(rate=0.2)
        self.layers2 = tf.keras.layers.Dense(units=16, activation ='relu')
        self.dropout2 = tf.keras.layers.Dropout(rate=0.2)
        self.layers3 = tf.keras.layers.Dense(units=2, activation='relu')

        self.final = tf.keras.layers.Dense(units=1, activation='sigmoid')

    def __repr__(self):
        return f"DeepFM Model: #Field {self.num_field}, #Feature {self.num_feature}, Embedding Size: {self.embedding_size}"

    def call(self, inputs):
        # 1) FM Component: (num_batch, 2)
        y_fm , new_inputs = self.fm_layer(inputs)

        # retrieve Dense Vectors: (num_batch,num_feature*embedding_size)
        new_inputs = tf.reshape(new_inputs, [-1,self.num_feature*self.embedding_size])

        # 2) Deep Component
        y_deep = self.layers1(new_inputs)
        y_deep = self.dropout1(y_deep)
        y_deep = self.layers2(y_deep)
        y_deep = self.dropout2(y_deep)
        y_deep = self.layers3(y_deep)

        # concatenation
        y_pred = tf.concat([y_fm, y_deep],1)
        y_pred = self.final(y_pred)
        y_pred= tf.reshape(y_pred, [-1,]) 

        return y_pred







### Train

In [ ]:
#import config
#from preprocess import get_modeified_data
#from DeepFM import DeepFM

import numpy as np
import pandas as pd
from time import perf_counter # returns the float value of time in seconds
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import BinaryAccuracy, AUC


In [ ]:
cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
data.target

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

In [ ]:
def get_data():
    X, Y = cancer, data.target
   
    
    X.columns = ALL_FIELDS
    field_dict, field_index, X_modified = get_modified_data(X, ALL_FIELDS,CONT_FIELDS, CAT_FIELDS, False)

    X_train, X_test, Y_train, Y_test = train_test_split(X_modified, Y, test_size=0.2, stratify=Y )

    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train.values, tf.float32), tf.cast(Y_train, tf.float32))) \
        .shuffle(500).batch(BATCH_SIZE)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test.values, tf.float32), tf.cast(Y_test, tf.float32))) \
        .shuffle(150).batch(BATCH_SIZE)

    return train_ds, test_ds, field_dict, field_index

In [ ]:
train_ds, test_ds, field_dict, field_index = get_data()
train_ds

Data Prepared...
X shape: (569, 30)
# of feature : 30
# of field: 30


<BatchDataset shapes: ((None, 30), (None,)), types: (tf.float32, tf.float32)>

In [ ]:
test_ds

<BatchDataset shapes: ((None, 30), (None,)), types: (tf.float32, tf.float32)>

In [ ]:
def get_data():
    file = pd.read_csv('data/adult.data', header=None)
    X = file.loc[:,0:13]
    Y = file.loc[:,14].map({'<=50K':0, '>50K':1})

    X.columns = config.ALL_FIELDS
    field_dict, field_index, X_modified = \ 
        get_modified_data(X, ALL_FIELDS, CONT_FIELDS, CAT_FIELDS, False)

    X_train, X_test, Y_train, Y_test = train_test_split( X_modified, Y, test_size=0.2, stratify=Y )

    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train.values, tf.float32), tf.cast(Y_train, tf.float32))) \
        .shuffle(500).batch(BATCH_SIZE)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test.values, tf.float32), tf.cast(Y_test, tf.float32))) \
        .shuffle(200).batch(BATCH_SIZE)

    return train_ds, test_ds, field_dict, field_index


In [ ]:
# Batch 단위 학습
def train_on_batch(model, optimizer, acc, auc, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False, y_true=targets, y_pred=y_pred)

    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용함
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # accuracy & auc 
    acc.update_state(targets, y_pred)
    auc.update_state(targets, y_pred)

    return loss


In [ ]:
# 반복 학습 함수
def train(epochs):
    train_ds , test_ds, field_dict, field_index = get_data()

    model = DeepFM(embedding_size=EMBEDDING_SIZE, num_feature = len(field_index),
                   num_field=len(field_dict), field_index=field_index)
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    print(f'Start Training: Batch Size: {BATCH_SIZE}, Embedding Size: {EMBEDDING_SIZE}')
    start = perf_counter()
    for i in range(epochs):
        acc = BinaryAccuracy(threshold=0.5)
        auc = AUC()
        loss_history = []

        for x,y in train_ds:
            loss = train_on_batch(model, optimizer, acc, auc, x, y)
            loss_history.append(loss)
        print(f'Epoch {i}: 누적 Loss: {np.mean(loss_history)}, ACC: {acc.result().numpy()}, AUC: {auc.result().numpy()}')

    test_acc = BinaryAccuracy(threshold=0.5)
    test_auc = AUC()
    for x,y in test_ds:
        y_pred = model(x)
        test_acc.update_state(y,y_pred)
        test_auc.update_state(y,y_pred)

    print(f'Test ACC: {test_acc.result().numpy()}, AUC: {test_auc.result().numpy()}')
    print(f'Batch size: {BATCH_SIZE}, Embedding size: {EMBEDDING_SIZE}')
    print(f'걸린 시간: {perf_counter()-start}')

    #model.save_weights(f'weights/weights-epoch({epochs})-batch({BATCH_SIZE})-embedding({EMBEDDING_SIZE}).h5')


if __name__ =='__main__':
    train(epochs=100)






Data Prepared...
X shape: (569, 30)
# of feature : 30
# of field: 30
Start Training: Batch Size: 30, Embedding Size: 5
Epoch 0: 누적 Loss: 1.1798657178878784, ACC: 0.6041666269302368, AUC: 0.07776058465242386
Epoch 1: 누적 Loss: 0.9826631546020508, ACC: 0.6250000596046448, AUC: 0.07717236131429672
Epoch 2: 누적 Loss: 0.8859766721725464, ACC: 0.6124999523162842, AUC: 0.07814241945743561
Epoch 3: 누적 Loss: 0.803745448589325, ACC: 0.6187500357627869, AUC: 0.08008256554603577
Epoch 4: 누적 Loss: 0.771704912185669, ACC: 0.6000000834465027, AUC: 0.08394221216440201
Epoch 5: 누적 Loss: 0.7378594279289246, ACC: 0.5666666626930237, AUC: 0.07888545840978622
Epoch 6: 누적 Loss: 0.7187329530715942, ACC: 0.5145832896232605, AUC: 0.0837048664689064
Epoch 7: 누적 Loss: 0.700771689414978, ACC: 0.5020833611488342, AUC: 0.15500517189502716
Epoch 8: 누적 Loss: 0.6905999779701233, ACC: 0.6645832657814026, AUC: 0.5450360774993896
Epoch 9: 누적 Loss: 0.6810152530670166, ACC: 0.8375000357627869, AUC: 0.9074612855911255
Epoch 1